In [172]:
import json
from glob import glob
import csv

In [156]:
def sort_by_line_number(elem):
    return elem['line_number']

def sort_by_weight(elem):
    return elem['weight']

In [87]:
input_file_pattern = 'zero-carbon-bill/input/*.json'
input_folder_name = 'zero-carbon-bill/input/'

filenames = glob(input_file_pattern)

In [198]:
# Import key phrases

key_phrases_file = 'zero-carbon-bill/key-phrases/key-phrases.json'
with open(key_phrases_file) as f:
    contents = f.read()

raw_key_phrases = json.loads(contents)
key_phrases = {}

for item in raw_key_phrases:
    filename = item['File']
    if key_phrases.get(filename, None) is None:
        key_phrases[filename] = []
    phrases = item['KeyPhrases']
    for phrase in phrases:
        phrase.pop('BeginOffset')
        phrase.pop('EndOffset')
    key_phrases[filename] = phrases

In [199]:
# Import sentiment analysis

sentiment_analysis_file = 'zero-carbon-bill/sentiment-analysis/output.json'
with open(sentiment_analysis_file) as f:
    contents = f.read()

raw_sentiment_analysis = json.loads(contents)
sentiment_analysis = {}

for item in raw_sentiment_analysis:
    filename = item['File']
    if sentiment_analysis.get(filename, None) is None:
        sentiment_analysis[filename] = []
    sentiment_analysis[filename].append(item)

In [200]:
# Import topic modelling: topics

topic_terms = {}

with open('zero-carbon-bill/topic-modelling/topic-terms.csv', newline='') as topic_terms_file:
    reader = csv.reader(topic_terms_file, delimiter=',')
    for row in reader:
        if row[0] == 'topic':
            next
        else:
            topic_number = row[0]
            item = {'term': row[1], 'weight': row[2]}
            if topic_terms.get(topic_number, None) is None:
                topic_terms[topic_number] = []
            topic_terms[topic_number].append(item)

for key, value in topic_terms.items():
    topic_terms[key] = sorted(value, key=sort_by_weight, reverse=True)

In [201]:
# Import topic modelling: documents

docs_with_topics = {}

with open('zero-carbon-bill/topic-modelling/doc-topics.csv', newline='') as doc_topics:
    reader = csv.reader(doc_topics, delimiter=',')
    for row in reader:
        if row[0] == 'docname':
            next
        else:
            filename = row[0]
            item = {'topic': row[1], 'confidence': row[2]}
            
            if docs_with_topics.get(filename, None) is None:
                docs_with_topics[filename] = []
            docs_with_topics[filename].append(item)

In [202]:
def key_phrases_for_submission(filename):
    if input_folder_name in filename:
        input_file_name = filename
        filename = filename.replace(input_folder_name, '')
    else:
        input_file_name = input_folder_name + filename

    p = key_phrases.get(filename, None)
    return(p)

In [203]:
def topics_for_submission(filename):
    if input_folder_name in filename:
        input_file_name = filename
        filename = filename.replace(input_folder_name, '')
    else:
        input_file_name = input_folder_name + filename

    return(docs_with_topics.get(filename, None))

In [204]:
def sentiment_analysis_for_submission(filename):    
    if input_folder_name in filename:
        input_file_name = filename
        filename = filename.replace(input_folder_name, '')
    else:
        input_file_name = input_folder_name + filename

    sentiment = sentiment_analysis[filename]
    
    with open(input_file_name) as f:
        submission_contents = f.readlines()

    labelled_line_numbers = []
    
    for l in sentiment:
        line_number = l['Line']
        
        sentiment_details = {}
        
        predicted_sentiment = l['Sentiment']
        sentiment_details['sentiment'] = predicted_sentiment.lower()
        sentiment_details['confidence'] = l['SentimentScore'][predicted_sentiment.title()]
                
        line_details = {}
        
        line_details['line_number'] = line_number
        line_details['original_text'] = submission_contents[line_number]
        line_details['sentiment'] = sentiment_details
        
        labelled_line_numbers.append(line_details)


    return(sorted(labelled_line_numbers, key=sort_by_line_number))

In [224]:
a = sentiment_analysis_for_submission(filenames[8])

print(filenames[8])
for line in a:
    print(line['original_text'])
    print(line['sentiment'])
    print('***')

zero-carbon-bill/input/09511_John_Adams_Redacted.json
As will be clear from the comments above - I am concerned that NZ has wasted almost a decade (with significant cumulative

{'sentiment': 'negative', 'confidence': 0.7689349055290222}
***


{'sentiment': 'neutral', 'confidence': 0.9320765733718872}
***
emissions in that time) because politics was allowed to get in the way of science. It is urgently time to turn that around and set up

{'sentiment': 'mixed', 'confidence': 0.9768466353416443}
***
systems that are distinctly science led. I am concerned that the conversation about co-benefits has not been well explored. At the

{'sentiment': 'neutral', 'confidence': 0.7430363297462463}
***
presentation in Hawke's Bay (and presumably elsewhere), the idea of land use change was put up as a "challenge" wheras we

{'sentiment': 'neutral', 'confidence': 0.9552146196365356}
***


{'sentiment': 'neutral', 'confidence': 0.9320765733718872}
***
already have a number of influential sectors and lea

In [207]:
def everything_for_file(filename):
    sentiment = sentiment_analysis_for_submission(filename)
    key_phrases = key_phrases_for_submission(filename)
    topics = topics_for_submission(filename)
    return({'sentiment': sentiment, 'key_phrases': key_phrases, 'topics': topics})

In [225]:
everything_for_file(filenames[8])

{'sentiment': [{'line_number': 0,
   'original_text': 'As will be clear from the comments above - I am concerned that NZ has wasted almost a decade (with significant cumulative\n',
   'sentiment': {'sentiment': 'negative', 'confidence': 0.7689349055290222}},
  {'line_number': 1,
   'original_text': '\n',
   'sentiment': {'sentiment': 'neutral', 'confidence': 0.9320765733718872}},
  {'line_number': 2,
   'original_text': 'emissions in that time) because politics was allowed to get in the way of science. It is urgently time to turn that around and set up\n',
   'sentiment': {'sentiment': 'mixed', 'confidence': 0.9768466353416443}},
  {'line_number': 3,
   'original_text': 'systems that are distinctly science led. I am concerned that the conversation about co-benefits has not been well explored. At the\n',
   'sentiment': {'sentiment': 'neutral', 'confidence': 0.7430363297462463}},
  {'line_number': 4,
   'original_text': 'presentation in Hawke\'s Bay (and presumably elsewhere), the idea 

In [226]:
topic_terms['000']

[{'term': 'future', 'weight': '0.052680425'},
 {'term': 'important', 'weight': '0.0458398'},
 {'term': 'work', 'weight': '0.04049393'},
 {'term': 'country', 'weight': '0.029050583'},
 {'term': 'green', 'weight': '0.029000932'},
 {'term': 'generation', 'weight': '0.021251926'},
 {'term': 'act', 'weight': '0.021079408'},
 {'term': 'policy', 'weight': '0.02003124'},
 {'term': 'clean', 'weight': '0.01826601'},
 {'term': 'kid', 'weight': '0.015781336'}]